In [1]:
import pandas as pd

# Pivoting a single variable

Supongamos que estamos comenzando a realizar un blog, la información respecta a este blog es la siguiente

In [6]:
users = pd.read_csv('users.csv')
users = users.drop('Unnamed: 0', axis = 1)
users

,weekday,city,visitors,signups
0,Sun,Austin,139,7
1,Sun,Dallas,237,12
2,Mon,Austin,326,3
3,Mon,Dallas,456,5


Supongamos que deseamos ver de una forma más clara el número de visitantes que tenemos por día de la semana y por ciudad. Para realizar estos contamos con la función **pivot**. Esta función recibe el argumento **index** donde indicamos la columna que queremos que actue como índice, el argumento **columns** que recible el nombre de la columna/s que queremos que se transforme en columna y la **values** que recibe el nombre de la columna/s que actuará como valor.

In [8]:
#Pivotamos nuestra tabla
df_pivot = users.pivot(index = 'weekday', columns = 'city', values = 'visitors')
df_pivot

city,Austin,Dallas
weekday,,
Mon,326,456
Sun,139,237


Podemos ver como de esta forma podemos saber el número de visitantes por ciudad y por día de la semana de una forma más clara.

# Pivoting all variables

Si en values, no asignamos ningún tipo de columna, todas las columnas que no se encuentren en index y columns serán pivotadas.

In [9]:
df_pivot = users.pivot(index = 'weekday', columns = 'city')
df_pivot

visitors        signups       
city      Austin Dallas  Austin Dallas
weekday                               
Mon          326    456       3      5
Sun          139    237       7     12

# Stacking & unstacking I

Los métodos **stack()** y **unstack()** nos permiten realizar una operación similar a **pivot()**.

In [5]:
#Cargamos los datos
users = pd.read_csv('users.csv', index_col = ['city', 'weekday'])

#Reordenamos los índices
users.sort_index(inplace = True)

#Vemos el resultado
users

Unnamed: 0  visitors  signups
city   weekday                               
Austin Mon               2       326        3
       Sun               0       139        7
Dallas Mon               3       456        5
       Sun               1       237       12

In [6]:
#Hacemos uso de unstack 
byweekday = users.unstack(level = 'weekday')

#Vemos el resultado
byweekday

Unnamed: 0     visitors      signups    
weekday        Mon Sun      Mon  Sun     Mon Sun
city                                            
Austin           2   0      326  139       3   7
Dallas           3   1      456  237       5  12

In [8]:
#Ahora hacemos uso de stack
byweekday.stack(level = 'weekday').drop('Unnamed: 0', axis = 1)

visitors  signups
city   weekday                   
Austin Mon           326        3
       Sun           139        7
Dallas Mon           456        5
       Sun           237       12

# Stacking & unstacking II

In [12]:
#Hacemos unstack por city
bycity = users.unstack(level ='city').drop('Unnamed: 0', axis = 1)
bycity

visitors        signups       
city      Austin Dallas  Austin Dallas
weekday                               
Mon          326    456       3      5
Sun          139    237       7     12

In [13]:
#Hacemos stack
bycity.stack(level = 'city')

signups  visitors
weekday city                     
Mon     Austin        3       326
        Dallas        5       456
Sun     Austin        7       139
        Dallas       12       237

# Restoring the index order

Existen situaciones en los que queremos cambiar el orden de los índices, para esto disponemos de **swaplevel()**.

In [40]:
#Aplicamos stack()
newusers = bycity.stack(level = 'city')

#Vemos el resultado
newusers

signups  visitors
weekday city                     
Mon     Austin        3       326
        Dallas        5       456
Sun     Austin        7       139
        Dallas       12       237

In [48]:
# Volteamos el indexado
newusers = newusers.swaplevel(0,1)

#Vemos el resultado
newusers

,,signups,visitors
city,weekday,,
Austin,Mon,3,326
Dallas,Mon,5,456
Austin,Sun,7,139
Dallas,Sun,12,237


# Adding names for readability

El objetivo de la función **pd.melt()** es retornar un dataframe a su forma original. Supongamos que disponemos del siguiente conjunto de datos.

In [3]:
#Cargamos los datos
users = pd.read_csv('users.csv')

#Pivotamos 
df_pivot = users.pivot(index = 'weekday', columns = 'city', values = 'visitors')

#Vemos el resultado
df_pivot

city,Austin,Dallas
weekday,,
Mon,326,456
Sun,139,237


Ahora queremos retorar este conjunto de datos a su forma original

In [4]:
#Hacemos uso de reset_index
df_pivot.reset_index(inplace=True)

#Vemos el resultado
df_pivot

city,weekday,Austin,Dallas
0,Mon,326,456
1,Sun,139,237


In [5]:
#Hacemos uso de melt
pd.melt(df_pivot, id_vars = ['weekday'], value_name = 'visitors' )

,weekday,city,visitors
0,Mon,Austin,326
1,Sun,Austin,139
2,Mon,Dallas,456
3,Sun,Dallas,237


# Going from wide to long

Podemos hacer que múltiples columnas esten agrupadas en una sola columna. Para ella indicamos en el argumento **id_vars** las columnas que no deseamos fusionar.

In [10]:
#Cargamos los datos
users = pd.read_csv('users.csv')

#Vemos el resultado 
users.drop('Unnamed: 0', axis = 1, inplace=True)
users

,weekday,city,visitors,signups
0,Sun,Austin,139,7
1,Sun,Dallas,237,12
2,Mon,Austin,326,3
3,Mon,Dallas,456,5


In [14]:
#Fusionamos las columnas visitors y signups en una misma
pd.melt(users, id_vars = ['weekday', 'city'])

,weekday,city,variable,value
0,Sun,Austin,visitors,139
1,Sun,Dallas,visitors,237
2,Mon,Austin,visitors,326
3,Mon,Dallas,visitors,456
4,Sun,Austin,signups,7
5,Sun,Dallas,signups,12
6,Mon,Austin,signups,3
7,Mon,Dallas,signups,5


# Obtaining key-value pairs with melt()

Existen situaciones en las que las que no estamos interesados en los índices pero si estamos interesados en los valores. 

In [18]:
#Cargamos los datos
users = pd.read_csv('users.csv')

#Vemos el resultado
users.drop('Unnamed: 0', axis = 1, inplace=True)
users

,weekday,city,visitors,signups
0,Sun,Austin,139,7
1,Sun,Dallas,237,12
2,Mon,Austin,326,3
3,Mon,Dallas,456,5


In [19]:
#Asignamos multi-índice
users = users.set_index(['city', 'weekday'])

#Vemos el resultado
users

,,visitors,signups
city,weekday,,
Austin,Sun,139,7
Dallas,Sun,237,12
Austin,Mon,326,3
Dallas,Mon,456,5


In [20]:
#Obtenemos los valores como  pareja clave valor
pd.melt(users, col_level = 0)

,variable,value
0,visitors,139
1,visitors,237
2,visitors,326
3,visitors,456
4,signups,7
5,signups,12
6,signups,3
7,signups,5


# Setting up a pivot table

Cuando tenems combinaciones repetidas para una misma pareja clave-valor el método **pivot** no funcionará. Para este tipo de situaciones el método **pivot_table** puede ser una muy buena opcion. Se trata de un método que nos permite agregar valores según determinadas funciones, por defecto hace uso de la media.

In [4]:
#Cargamos los datos
users = pd.read_csv('users.csv')
users.drop('Unnamed: 0', axis = 1, inplace = True)

by_city_day = users.pivot_table(index = 'weekday', columns = 'city')

#Vemos el resultado
by_city_day

signups        visitors       
city     Austin Dallas   Austin Dallas
weekday                               
Mon           3      5      326    456
Sun           7     12      139    237

# Using other aggregations in pivot tables

Como ya hemos indicado podemos hacer agregaciones, para ello contamos con el parámetro **aggfunc**.

In [7]:
#Agregamos por la función media
users.pivot_table(index = 'weekday')

,signups,visitors
weekday,,
Mon,4.0,391.0
Sun,9.5,188.0


In [9]:
#Agregamos por la función media
users.pivot_table(index = 'weekday', aggfunc = 'count')

,city,signups,visitors
weekday,,,
Mon,2,2,2
Sun,2,2,2


# Using margins in pivot tables

Es posible conocer el total haciendo uso del parámetro **margins**, para esto tan solo debemos darle el valor **True** a dicho parámetro.

In [10]:
#Calculamos el número de usuarios y registros por día
users.pivot_table(index = 'weekday', aggfunc = 'sum')

,signups,visitors
weekday,,
Mon,8,782
Sun,19,376


In [11]:
#Ahora hacemos uso de margins para obtener el total 
users.pivot_table(index = 'weekday', aggfunc = 'sum', margins = True)

,signups,visitors
weekday,,
Mon,8.0,782.0
Sun,19.0,376.0
All,27.0,1158.0
